# comparisons

This is a text cell. Start editing!

In [1]:
import org.apache.spark._
import org.apache.spark.sql.types._
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._
import group.research.aging.spark.extensions._
import group.research.aging.spark.extensions.functions._
import kernel.display.html

In [2]:
 
  val expressionsPath = "/data/results/gray-whale/Expressions"
  val byGoPath = expressionsPath + "/GO"
  val comparison = byGoPath + "/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv"
  val grouped = byGoPath + "/grouped"
  val ranked = byGoPath + "/grouped/ranked"

In [5]:
val batGO = spark.readTSV("/data/results/gray-whale/Expressions/GO/raw/bat_go_all.tab", true)
batGO.show(10,1000)

+----------+------------------+------------------+------------------+------------------+
|        go|       bat_liver_1|      bat_kidney_1|       bat_liver_2|      bat_kidney_2|
+----------+------------------+------------------+------------------+------------------+
|GO:0005615| 91336.02574200001| 3421.090592000001| 90348.99755000009|3687.9415869999993|
|GO:0005737| 35870.69648499997|25779.108638999987| 34057.78138999998| 25298.75620900001|
|GO:0016021|31293.667532000018| 35415.92437499999|26379.918117000005| 36627.32599800002|
|GO:0016020|28317.733255000006| 21595.71151800001| 27932.64955000001|      20759.532237|
|GO:0005634|22345.484704000002|29520.300637000008|20232.025698000016|28844.954576000004|
|GO:0005829|20550.202911000008|21499.736783999993| 18300.47772899999|21683.063729999994|
|GO:0005576|20491.114166999992| 6261.713387999997| 19631.34355300001| 5706.305368999997|
|GO:0005829|17416.220077999995|      24845.416096| 16745.21679700001| 23481.33331900002|
|GO:0005524|16435.664

In [8]:
(batGO.count, batGO.distinct.count, batGO.select("go").distinct.count) //WHAT THE FUCK?

(13694,13602,12256)

In [7]:
val goTable = spark.readTSV(comparison, true)
goTable.show(10,1000)

+----------+------------------+----------------------------------+------------------+--------------+------------+-----------------+-----------------------+-----------------------+---------------------+-----------------------------------------------+------------------+-------------------+------------------+------------------+------------------+--------------------+------------------+------------------+-----------------+------------------+--------------------+------------------+------------------+------------------+---------------------+------------------+-------------------+------------------+
|        go|         namespace|                             label|    gray_whale_avg|uniref90_count|taxons_count|transcripts_count|human_transcripts_count|mouse_transcripts_count|cow_transcripts_count|                                           type|  gray_whale_liver|bowhead_whale_liver|       minke_liver|         NMR_liver|       human_liver|mouse_totalRNA_liver|  mouse_mRNA_liver|cow_totalRNA_li

In [10]:
goTable.columns.toList.mkString("""Seq("""", """","""", """")""")

Seq("go","namespace","label","gray_whale_avg","uniref90_count","taxons_count","transcripts_count","human_transcripts_count","mouse_transcripts_count","cow_transcripts_count","type","gray_whale_liver","bowhead_whale_liver","minke_liver","NMR_liver","human

In [6]:
val goTableUpd = goTable.join(batGO, Seq("go"), "outer").select(
   "go","namespace","label","gray_whale_avg","uniref90_count","taxons_count","transcripts_count","human_transcripts_count","mouse_transcripts_count","cow_transcripts_count","type",
    "gray_whale_liver","bowhead_whale_liver","minke_liver","NMR_liver","human_liver","mouse_totalRNA_liver","mouse_mRNA_liver","cow_totalRNA_liver","cow_mRNA_liver", "bat_liver_1", "bat_liver_2",
    "gray_whale_kidney","bowhead_whale_kidney","minke_kidney","NMR_kidney","human_kidney","mouse_totalRNA_kidney","mouse_mRNA_kidney","cow_totalRNA_kidney","cow_mRNA_kidney","bat_kidney_1", "bat_kidney_2")
goTableUpd.show(10,1000)


+----------+------------------+---------------------------------------------------------+------------------+--------------+------------+-----------------+-----------------------+-----------------------+---------------------+-----------------------------------------------+------------------+-------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+---------------------+------------------+-------------------+-----------------+------------------+------------------+
|        go|         namespace|                                                    label|    gray_whale_avg|uniref90_count|taxons_count|transcripts_count|human_transcripts_count|mouse_transcripts_count|cow_transcripts_count|                                           type|  gray_whale_liver|bowhead_wh

In [9]:
  val componentsUpd = goTableUpd.where("namespace = 'cellular_component'")
  val processesUpd = goTableUpd.where("namespace = 'biological_process'")
  val functionsUpd = goTableUpd.where("namespace = 'molecular_function'")

  (componentsUpd.count, processesUpd.count, functionsUpd.count, goTableUpd.count)

(2347,12638,4934,20505)

In [12]:
componentsUpd.writeTSV(grouped + "/updated/by_cellular_component.tab", true)
processesUpd.writeTSV(grouped + "/updated/by_biological_process.tab", true)
functionsUpd.writeTSV(grouped + "/updated/by_molecular_function.tab", true)


parts of /data/results/gray-whale/Expressions/GO/grouped/updated/by_cellular_component.tab merged!
parts of /data/results/gray-whale/Expressions/GO/grouped/updated/by_biological_process.tab merged!
parts of /data/results/gray-whale/Expressions/GO/grouped/updated/by_molecular_function.tab merged!


/data/results/gray-whale/Expressions/GO/grouped/updated/by_molecular_function.tab

In [13]:
 def rank(df: DataFrame, name: String, rankSuffix: String = "_rank") =
    df.withColumn(name + rankSuffix, org.apache.spark.sql.functions.dense_rank().over(Window.orderBy(new ColumnName(name).desc)))

  def ranks(df: DataFrame,
            names: Seq[String],
            rankSuffix: String = "_rank") = names.foldLeft(df){
    case (f, n)=> rank(f, n, rankSuffix)
  }

  val rankedCols = componentsUpd.columns.drop(11)++List("gray_whale_avg")

val rankedComponents = ranks(componentsUpd, rankedCols)
val rankedProcesses = ranks(processesUpd, rankedCols)
val rankedFunctions = ranks(functionsUpd, rankedCols)

In [16]:
rankedProcesses.show()

+----------+------------------+--------------------+------------------+--------------+------------+-----------------+-----------------------+-----------------------+---------------------+--------------------+------------------+-------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+---------------------+------------------+-------------------+------------------+------------------+------------------+---------------------+------------------------+----------------+--------------+----------------+-------------------------+---------------------+-----------------------+-------------------+----------------+----------------+----------------------+-------------------------+-----------------+---------------+-----------------+--------------------------+---------------

In [14]:
rankedComponents.writeTSV(grouped + "/updated/by_cellular_component_ranked.tab", true)
rankedProcesses .writeTSV(grouped + "/updated/by_biological_process_ranked.tab", true)
rankedFunctions.writeTSV(grouped + "/updated/by_molecular_function_ranked.tab", true)


parts of /data/results/gray-whale/Expressions/GO/grouped/updated/by_cellular_component_ranked.tab merged!
parts of /data/results/gray-whale/Expressions/GO/grouped/updated/by_biological_process_ranked.tab merged!
parts of /data/results/gray-whale/Expressions/GO/grouped/updated/by_molecular_function_ranked.tab merged!


/data/results/gray-whale/Expressions/GO/grouped/updated/by_molecular_function_ranked.tab